# Six-bar Mechanism Balancing

In [1]:
from BetaShF import ShF
from BetaShM import ShM 
import numpy as np 
from scipy.optimize import differential_evolution, minimize
import matplotlib.pyplot as plt
from cnsg_differential_evolution import cnsg_differential_evolution
import time

### Utils

In [2]:
def cleanData(samples, fitness, forces, moments):
    filterF = forces < 1
    filterM = moments < 1
    f = np.logical_and(filterF, filterM)
    print(f.shape)
    return samples[f], fitness[f], forces[f], moments[f]

In [3]:
def logSample(now, sample, fitness, force, moment):
    def appendToFile(name, text): 
        with open(name, "a") as f:
            f.write(text + '\n')
    s = ""
    for x in sample: s += str(x) + " "
    appendToFile(now + "Population.txt", s)
    appendToFile(now + "Fitness.txt", str(fitness))
    appendToFile(now + "ShForces.txt", str(force))
    appendToFile(now + "ShMoments.txt", str(moment))

### Problem Definition

##### Contraints

$$-0.16m <= x_{cn},y_{cn} <= 0.16m$$

$$0.005m <= t_{cn} <= 0.04m$$


##### Objective Function

In [9]:
def objective_function(s, ShF, ShM, a): #c is a constant that distributes the weight among the functions.
    return a*ShF(s) + (1-a)*ShM(s)

### Define boundaries

In [10]:
# Bounds for each variable
nVar = 5
bounds = []
for i in range(1,nVar*3+1):
    if(i%3==0): bounds.append([0.005,0.04])
    else: bounds.append([-0.16, 0.16])
bounds = np.array(bounds)
print('bounds',bounds)

bounds [[-0.16   0.16 ]
 [-0.16   0.16 ]
 [ 0.005  0.04 ]
 [-0.16   0.16 ]
 [-0.16   0.16 ]
 [ 0.005  0.04 ]
 [-0.16   0.16 ]
 [-0.16   0.16 ]
 [ 0.005  0.04 ]
 [-0.16   0.16 ]
 [-0.16   0.16 ]
 [ 0.005  0.04 ]
 [-0.16   0.16 ]
 [-0.16   0.16 ]
 [ 0.005  0.04 ]]


## Gradient Descent

### CG

In [11]:
def gradiente_conjugado(X0,f,MaxIter=100,eps=1e-5):
    k=0
    X = X0
    G = Vf(X,ShF, ShM, f)
    normaGradiente = np.linalg.norm(G)
    P = -G
    curr_fit = f(X,ShF, ShM)
    while(k<=MaxIter and normaGradiente>=eps):
        Ap = V2fTd(X,ShF, ShM,P,f)
        alpha = np.dot(-P,G) / np.dot(P,Ap)
        X = X + alpha*P
        G_ = np.copy(G)
        G = G + alpha*Ap
        normaGradiente = np.linalg.norm(G)
        B = -np.dot(G,G)/np.dot(G_,G_)
        P = -G + B*P
        k = k+1
        print("#",k, ", fit: ", f(X,ShF, ShM))
        if f(X,ShF, ShM) <= curr_fit:
            best_x = X
            curr_fit = f(best_x,ShF, ShM)
    return best_x

### GD

In [12]:
import numpy as np
#################### Descenso de Gradiente con diferenciación finita
eps = 1e-5
def Gradiente(X,f, ShF, ShM):
    n = len(X)
    G = np.zeros((n),float)
    incX = np.zeros((n),float)
    for i in range(n):
        incX[i] = eps
        G[i] = (f(X+incX, ShF, ShM)-f(X, ShF, ShM))/eps
        incX[i] = 0
    return G
def getStepSize(a,m,X,P,G,f):
    c0 = 1e-4
    c1 = 2
    c2 = 5
    c3 = 3
    eps = 1e-8
    alpha = a
    while f(X+alpha*P, ShF, ShM) > f(X, ShF, ShM)+c0*np.dot(G,P):
        m = 0
        alpha = alpha/c1
        if alpha<=eps:
            break
    m += 1
    if m>=c2:
        m=0
        alpha = c3*alpha
    return alpha,m

def Gradient_Descent(X0,f,bounds,MaxIter=1000,alpha=1e-3):
    k=0
    X = X0
    G = Gradiente(X,f, ShF, ShM)
    normaGradiente = np.linalg.norm(G)
    m = 0
    while(k<=MaxIter and normaGradiente>=eps):
        G = Gradiente(X,f, ShF, ShM)
        normaGradiente = np.linalg.norm(G)
        P = - G / normaGradiente
        alpha,m = getStepSize(alpha,m,X,P,G,f)
        X = X + alpha*P
        X = np.clip(X,bounds[:,0],bounds[:,1])
        # verficar bounds: si x() no coincide con sus bounds correspondientes
        k = k+1
        if k%100 == 0:
            print("\t\t#",k,f(X, ShF, ShM))
    return X


In [13]:
s=bounds[:,0]
r = objective_function(s, ShF, ShM,0.5)
print('OF:',r)

OF: 46.604883632565645


In [14]:
def random_start(bounds):
    # print(bounds.shape[0])
    arr = np.zeros(bounds.shape[0])
    for i, tupl in enumerate(bounds):
        rand = np.random.uniform(tupl[0], tupl[1])
        # print(i, rand)
        arr[i] = rand 
    return arr
    
print(random_start(bounds))

[ 0.00052678 -0.09033851  0.00938731  0.06676138  0.09533738  0.0320544
 -0.10050153  0.11142163  0.00951557 -0.09210145  0.1485086   0.03725161
 -0.07886071 -0.09890708  0.01401429]


### Gradient Descent

In [15]:
# it = 80
# n = 4

def GD(alph, iter):
    n = 4
    sols = np.zeros((n, 2))
    best, bestSol = 10, None
    eTime = 0
    print(f'\n* Number of iterations: {iter}')
    for i in range(n):
        r = random_start(bounds)
        fitness = objective_function(r, ShF, ShM, 0.5)
        print(f"\t\nInitial Fitness: {fitness} in #{i}")
        start = time.perf_counter()
        r = Gradient_Descent(r,objective_function, bounds,MaxIter=iter, alpha=alph)
        end = time.perf_counter()
        fit_GD = objective_function(r,ShF,ShM, 0.5)
        print("\t  - before: ", fitness)
        print("\t  - after (GD): ", fit_GD)
        if fit_GD < best:
            best = fit_GD
            bestSol = r
        eTime += (end-start) #Time in seconds
    if n: eTime /= n
    print("Average time of execution:", eTime,"seconds. It was run", n, "times.")
    # return fit_GD

it = 700
print(GD(1e-3, it))
print(GD(1e-5, it))
print(GD(1e-8, it))


* Number of iterations: 700
	
Initial Fitness: 22.56949888982971 in #0
		# 100 0.5599638725626395
		# 200 0.4955442996748294
		# 300 0.4945356349259049
		# 400 0.4946503741589774
		# 500 0.49570697898223576
		# 600 0.49445754795343955
		# 700 0.49492465004120734
	  - before:  22.56949888982971
	  - after (GD):  0.49470318195097307
	
Initial Fitness: 47.59492041553046 in #1
		# 100 0.6084312705358962
		# 200 0.49751771546159956
		# 300 0.49573106819531654
		# 400 0.4944346481723204
		# 500 0.49472931303520395
		# 600 0.49455973942652737
		# 700 0.4944349955933619
	  - before:  47.59492041553046
	  - after (GD):  0.4944429173428567
	
Initial Fitness: 18.189827936872323 in #2
		# 100 0.6454546267054095
		# 200 0.5127697908245381
		# 300 0.4946284797863457
		# 400 0.4945968471171239
		# 500 0.49469326915338674
		# 600 0.49442428487387124
		# 700 0.4944256982854667
	  - before:  18.189827936872323
	  - after (GD):  0.4944252270295144
	
Initial Fitness: 43.37956367522239 in #3
		# 100 0.794